# Лабораторная работа №4

### Реализовать следующие структуры данных:

Стек (stack):
- операции для стека:
- инициализация;
- проверка на пустоту;
- добавление нового элемента в начало;
- извлечение элемента из начала;

Дек (двусторонняя очередь, deque):
- операции для дека:
- инициализация;
- проверка на пустоту;
- добавление нового элемента в начало;
- добавление нового элемента в конец;
- извлечение элемента из начала;
- извлечение элемента из конца;

Разработать программу обработки данных, содержащихся в заранее подготовленном txt-файле, в соответствии с заданиями, применив указанную в задании структуру данных. Результат работы программы вывести на экран и сохранить в отдельном txt-файле.

### Stack

In [3]:
import copy

In [4]:
class Stack:
    
    def __init__(self, size):
        # Создаем массив
        self.arr = ['empty'] * size
        # Кол-во занятых переменных
        self.count = 0
        
    # Добавление нового элемента
    def push(self, elem):
        if self.count + 1 > len(self.arr): 
            raise Exception('Элемент {}, не может быть помещен в Stack, так как он будет переполнен.'.format(elem))
        self.arr[self.count] = elem
        self.count += 1
        
    # Извлечение элемента
    def pop(self):
        if self.count - 1 >= 0:
            elem = self.arr[self.count-1]
            self.arr[self.count-1] = 'empty'
            self.count -= 1
            if elem == None:
                return 0
            else:
                return elem
            
    def top(self):
        elem = self.arr[self.count-1]
        if elem == None:
            return 0
        else:
            return elem
                    
    def isEmpty(self):
        if self.count == 0:
                return True        
        return False

    def print(self):
        print("Stack =", self.arr,"len =", self.count)

### Deque

In [5]:
class Deque(Stack):
    # Инициализатор
    def __init__(self, size):
        self.j = -1
        Stack.__init__(self, size)
    
    # Сдвигаем все элементы массива в нужную сторону
    def _move(self, direct):
        if direct == 'right' and self.arr[len(self.arr) - 1] == 'empty':
            nArr = ['empty'] * len(self.arr)
            j = 0
            for i in range(len(self.arr)):
                if i >= len(self.arr) - self.count:
                    nArr[i] = self.arr[j]
                    j += 1
            self.arr = nArr.copy()
        if direct == 'left' and self.arr[0] == 'empty':
            nArr = ['empty'] * len(self.arr)
            j = len(self.arr) - self.count
            for i in range(len(self.arr)):
                if j < len(self.arr):
                    nArr[i] = self.arr[j]
                    j += 1
            self.arr = nArr.copy()
            
    # Добавление нового элемента в начало
    def push(self, elem):
        Deque._move(self, 'left')
        Stack.push(self, elem)
        
    # Извлечение элемента с начала    
    def pop(self):
        Deque._move(self, 'left')
        return Stack.pop(self)
        
    # Добавление нового элемента в конец
    def pushEnd(self, elem):
        if len(self.arr) < 2:
            Stack.push(self, elem)
        if self.count + 1 > len(self.arr):
            raise Exception('Элемент {}, не может быть помещен в Stack, так как он будет переполнен.'.format(elem))
        else:
            Deque._move(self, 'right')
            self.arr[len(self.arr) - self.count - 1] = elem
            self.count += 1
            
    # Извлечение элемента с конца
    def popEnd(self):
        if len(self.arr) - (self.count - 1) <= len(self.arr):
            Deque._move(self, 'right')
            elem = self.arr[len(self.arr) - self.count] 
            self.arr[len(self.arr) - self.count] = 'empty'
            self.count -= 1
            return elem

### Задание 1

Отсортировать строки файла, содержащие названия книг, в алфавитном порядке с использованием 2-ух деков.

In [33]:
books = []

# Удаление символов переноса строки
def nlDel(s):
    pos = s.find('\n')
    if pos > -1:
        return s[0:pos]
    else:
        return s

# Получаем список книг и записываем его в лист Books 
f = open('task1/input.txt', encoding="utf8")
for line in f:
    books.append(nlDel(line))
f.close()
    
# Создаем две деки
d1 = Deque(len(books))
d2 = Deque(len(books))

# Заполняем первую Деку книгами из списка
for book in range(len(books)):
    d1.push(books[book])

def sort(fr, to): 
    # Перекидываем первый элемент из одной деки в другую
    book = fr.pop()
    lastBook = book
    to.push(book)
    
    # Сортируем оставшиеся элементы
    for i in range(len(books)-1):
        for j in range(min(len(lastBook),len(book))):
            book = fr.pop()
            if ord(lastBook[j]) != ord(book[j]):
                if ord(lastBook[j]) < ord(book[j]):
                    to.pushEnd(book)
                if ord(lastBook[j]) > ord(book[j]):
                    to.push(book)
                lastBook = book
                break
            else:
                fr.push(book)
                
    # Проверяем, что все элементы отсортированы
    isSorted = True
    book = to.pop()
    lastBook = book
    fr.push(book)
    
    for i in range(len(books)-1):
        for j in range(min(len(lastBook),len(book))):
            book = to.pop()
            if ord(lastBook[j]) != ord(book[j]):
                if ord(lastBook[j]) > ord(book[j]):
                    isSorted = False
                fr.pushEnd(book)
                lastBook = book
                break
            else:
                to.push(book)
        
    if (not isSorted):
        return sort(fr,to)
    
# Сортируем список книг
sort(d1,d2)

# Открываем файл для записи 
f = open('task1/output.txt', 'w', encoding="utf8")
# Заносим отсортированные книги в файл
for i in range(d1.count):
    print(d1.pop(), file=f)
f.close()

### Задание 2

Дек содержит последовательность символов для шифровки сообщений. Дан текстовый файл, содержащий зашифрованное сообщение. Пользуясь декой нужно, расшифровать текст. Известно, что при шифровке каждый символ сообщения заменялся следующим за ним в деке по часовой стрелке через один

In [7]:
import random

In [8]:
# Создаем словарь заменяемых символов и перемешиваем список этих символов
alphabet = list('АаБбВвГгДдЕеЁёЖжЗзИиЙйКкЛлМмНнОоПпСсТтУуФфХхЦцЧчШшЩщЪъЫыЬьЭэЮюЯя-.?!)(,:\\\'"')
random.shuffle(alphabet)
alphabet = ''.join(alphabet)

# Заполням дек
d = Deque(len(alphabet))
[d.push(alphabet[i]) for i in range(len(alphabet))]

# Строка для шифрования
s = 'Толпа лягушек на болоте.'

# Функция для шифрования
def encode(t,d):
    t = list(t)
    # Проходим все символы в исходной строке
    for i in range(len(t)):
        for j in range(d.count):
            c = d.pop()
            # Если указанный символ есть в нашем списке символов, то
            # заменяем символ из строки следующим через 1 символ из списка
            if c == t[i]:
                d.pushEnd(c)
                d.pushEnd(d.pop())
                c = d.pop()
                t[i] = c
            d.pushEnd(c)
    return ''.join(t)
   
# Функция для расшифровки    
def decode(t,d):
    t = list(t)
    # Проходим все символы в исходной строке
    for i in range(len(t)):
        for j in range(d.count):
            c = d.pop()
            # Если указанный символ есть в нашем списке символов, то
            # заменяем символ из строки следующим через 1 символ из списка (в обратном порядке)
            if c == t[i]:
                d.push(c)
                d.push(d.popEnd())
                c = d.popEnd()
                t[i] = c
            d.push(c)
    return ''.join(t)

print(encode(s,d), "- Зашифровонная фраза",)
print(decode(s,d), "- Расшифровка")

.ПШтГ ШуаиЩэЕ )Г фПШПЖэЧ - Зашифровонная фраза
Толпа лягушек на болоте. - Расшифровка


### Задание 3

Даны три стержня и n дисков различного размера. Диски можно надевать на стержни, образуя из них башни. Перенести n дисков со стержня А на стержень С, сохранив их первоначальный порядок.

При переносе дисков необходимо соблюдать следующие правила:

- на каждом шаге со стержня на стержень переносить только один диск;
- диск нельзя помещать на диск меньшего размера;
- для промежуточного хранения можно использовать стержень В.

Реализовать алгоритм, используя три стека вместо стержней А, В, С. Информация о дисках хранится в исходном файле.


In [10]:
disks = 3
# Создаем три стека для стержней равные кол-ву дисков
A = Stack(disks) 
B = Stack(disks) 
C = Stack(disks) 

# Заполняем первый стержень всеми дисками
for i in range(disks, 0, -1):
    A.push(i)
    
print('A')
A.print()

print('--- Перемещения ---')

# Основная функция, с помощью которой размещаем диски на стержнях
def move(st1, st2, let1, let2):
    if st1.count == 0 and st2.count > 0:
        st1.push(st2.pop())
    elif st1.count > 0 and st2.count == 0:
        st2.push(st1.pop())
    elif st1.top() > st2.top():
        st1.push(st2.pop())
    else:
        st2.push(st1.pop())
    
    print('\n',let1)
    st1.print()
    print(let2)
    st2.print()
    
if disks % 2 == 0:
    while C.count != disks:
        move(A, B, 'A', 'B')
        move(A, C, 'A', 'C')
        move(B, C, 'B', 'C')
else:
    while C.count != disks:
        move(A, C, 'A', 'C')
        if C.count == disks:
            break
        move(A, B, 'A', 'B')
        move(B, C, 'B', 'C')

print('--- Итог ---')
print('C')
C.print()

A
Stack = [3, 2, 1] len = 3
--- Перемещения ---

 A
Stack = [3, 2, 'empty'] len = 2
C
Stack = [1, 'empty', 'empty'] len = 1

 A
Stack = [3, 'empty', 'empty'] len = 1
B
Stack = [2, 'empty', 'empty'] len = 1

 B
Stack = [2, 1, 'empty'] len = 2
C
Stack = ['empty', 'empty', 'empty'] len = 0

 A
Stack = ['empty', 'empty', 'empty'] len = 0
C
Stack = [3, 'empty', 'empty'] len = 1

 A
Stack = [1, 'empty', 'empty'] len = 1
B
Stack = [2, 'empty', 'empty'] len = 1

 B
Stack = ['empty', 'empty', 'empty'] len = 0
C
Stack = [3, 2, 'empty'] len = 2

 A
Stack = ['empty', 'empty', 'empty'] len = 0
C
Stack = [3, 2, 1] len = 3
--- Итог ---
C
Stack = [3, 2, 1] len = 3


### Задание 4

Дан текстовый файл с программой на алгоритмическом языке. За один просмотр файла проверить баланс круглых скобок в тексте, используя стек.

In [9]:
# Основная функция
def isBracketsBalance(s):
    # Если число скобок нечетное, сразу возвращаем результат
    if len(s) % 2 != 0:
        return False
    
    # Создаем Stack размером с исходную строку
    # будем харнить в нем все открывающиеся скобки
    st = Stack(len(s))
    
    for c in s:
        if c == '(':
            st.push(c)
        # Если встретили закрывающую скобку, то проверяем не пуст ли Stack 
        # и если это не так, то удаляем скобку в начале списка
        if c == ')':
            if st.isEmpty():
                return False
            st.pop()
            
    return st.isEmpty()

print(isBracketsBalance('()('))
print(isBracketsBalance('()())((())(()(())()'))
print(isBracketsBalance('(()())()()()()(()(()(())()))'))
print(isBracketsBalance('))(('))

False
False
True
False


### Задание 5

Дан текстовый файл с программой на алгоритмическом языке. За один просмотр файла проверить баланс квадратных скобок в тексте, используя дек.

In [11]:
# Основная функция
def isBracketsBalance(s):
    # Если число скобок нечетное, сразу возвращаем результат
    if len(s) % 2 != 0:
        return False
    
    # Создаем Deque размером с исходную строку
    # будем харнить в нем все открывающиеся скобки.
    d = Deque(len(s))
    
    # Начинаем перебирать строку со скобками
    for c in s:
        if c == '[':
            d.push(c)
        # Если встретили закрывающую скобку, то проверяем не пуст ли Deque 
        # и если это не так, то удаляем скобку в начале списка
        if c == ']':
            if d.isEmpty():
                return False
            d.pop()
            
    # Возващяем boolean значение "пуст ли Deque"
    return d.isEmpty()

print(isBracketsBalance('[[]'))
print(isBracketsBalance('[][[][[[[]][[]]]]]]'))
print(isBracketsBalance('[[][][][[][]][][[][][][][]]][]'))
print(isBracketsBalance('][][[]'))

False
False
True
False


### Задание 6

Дан файл из символов. Используя стек, за один просмотр файла напечатать сначала все цифры, затем все буквы, и, наконец, все остальные символы, сохраняя исходный порядок в каждой группе символов.

In [34]:
t = ''

symbs = ''
nums = ''
lets = ''

f = open('task6/text.txt', encoding="utf8")
for line in f:
    t += line
f.close()

# Создаем стек и заполняем его всеми символами подряд в обратном порядке
st = Stack(len(t))
for c in reversed(t):
    st.push(c)

# Проходим по всему стеку ОДИН РАЗ и сравниваем коды символов с необходимыми, 
# помещаем символы в соответствующие массивы
for i in range(len(t)):
    # Если символ это цифра
    if ord(st.top()) >= 48 and ord(st.top()) < 58:
        nums += st.pop()
    # Если это спец. символ
    elif ord(st.top()) >= 33 and ord(st.top()) < 48 or ord(st.top()) >= 58 and ord(st.top()) < 65:
        symbs += st.pop()
    # Если это пробел
    elif ord(st.top()) == 32:
        if len(symbs)-1 > -1 and ord(symbs[len(symbs)-1]) != 32:
            symbs += ' '
        if len(nums)-1 > -1 and ord(nums[len(nums)-1]) != 32:
            nums += ' '
        if len(lets)-1 > -1 and ord(lets[len(lets)-1]) != 32:
            lets += ' '
        st.pop()
    # Все остальное
    else:
        lets += st.pop()
        
print(nums)
print(lets)
print(symbs)

1678 
В селе располагалась усадьба помещиков Кондыревых переселившихся сюда из Рязанской губернии

Так как практически все рязанские землевладельцы имели поместья на юговосточных рубежах которые необходимо было защищать от набегов степняков то Алексинский уезд при его освоении заселялся в основном рязанскими помещиками По всей видимости из родового гнезда Кондыревых — села Дубровичи при речке Ерастовке Рязанской губернии сюда переселился один из рязанских землевладельцев Фёдор Степанович Кондырев

В писцовых книгах Алексинского уезда за год есть запись «Стан Березовский за помещики… За Федором Степановым сыном Кондыревым жеребей села Березова двор крестьянской двор бобыльской двор задворного человека…»
, . - , , , . , . : , , , .


### Задание 7

Дан файл из целых чисел. Используя дек, за один просмотр файла напечатать сначала все отрицательные числа, затем все положительные числа, сохраняя исходный порядок в каждой группе

In [35]:
t = ''

pos = ''
neg = ''

# Записываем весь текст из файла в переменную
f = open('task7/nums.txt', encoding="utf8")
for line in f:
    t += line
f.close()

# Создаем стек и заполняем его всеми символами подряд из файла в обратном порядке
st = Stack(len(t))
for c in reversed(t):
    st.push(c)

isNegAdd = False
# Проходим по всему стеку ОДИН РАЗ и сравниваем коды символов с необходимыми, 
# помещаем символы в соответствующие массивы
for i in range(len(t)):
    # Если это пробел
    if ord(st.top()) == 32:
        isNegAdd = False
        if len(pos)-1 > -1 and ord(pos[len(pos)-1]) != 32:
            pos += ' '
        if len(neg)-1 > -1 and ord(neg[len(neg)-1]) != 32:
            neg += ' '
        st.pop()
    # Если это минус (-)
    elif ord(st.top()) == 45:
        isNegAdd = True
        neg += st.pop()
    else:
        if isNegAdd:
            neg += st.pop()
        else:
            pos += st.pop()
            
print("Отрицательные числа:", neg)
print("Положительные числа:", pos)

Отрицательные числа: -43 -31 -3235 -325 -5467 -5 -89 -34 -4859 -39 
Положительные числа: 5 8 293825 5431 335 4374 7847 7859 7 9 6273 


### Задание 8

Дан текстовый файл. Используя стек сформировать новый текстовый файл, содержащий строки исходного файла, записанные в обратном порядке: первая строка становится последней, вторая – предпоследней и т.д.

In [36]:
# Функция для удаления символов переноса строки
def nlDel(s):
    pos = s.find('\n')
    if pos > -1:
        return s[0:pos]
    else:
        return s

books = []
    
# Открываем файл для записи
f = open('task1/output.txt', encoding="utf8")
# Копируем книги из файла в список "Books"
for line in f:
    books.append(nlDel(line))
f.close()

s = Stack(len(books))

# Заполняем Stack
for i in range(len(books)):
    s.push(books[i])

# Открываем файл для записи 
f = open('task8/output-reverse.txt', 'w', encoding="utf8")
# Заносим отсортированные книги в файл
for i in range(s.count):
    print(s.pop(), file=f)
f.close()

### Задание 9

Дан текстовый файл. Используя Stack, вычислить значение логического выражения, записанного в текстовом файле в следующей форме:

< ЛВ > ::= T | F | (N<ЛВ>) | (<ЛВ>A<ЛВ>) | (<ЛВ>X<ЛВ>) | (<ЛВ>O<ЛВ>)

где буквами обозначены логические константы и операции:

- T – True
- F – False
- N – Not
- A – And
- X – Xor
- O – Or

In [13]:
class findLg:
    def __init__(self, s):
        # Стек, в котором в дальнейшем происходит работа со строкой
        st = Stack(len(s))
        
        # Заполнение стека по-символьно содержимым строки
        for c in reversed(s):
            st.push(c)
        # В переменную res заносим результат, а исходную строку помещаем в s
        self.res = self.__scan(st)
        self.s = s
        
    # Основной метод
    def __scan(self, st):
        if (st.top() == 'T'):
            st.pop()
            return True
        elif (st.top() == 'F'):
            st.pop()
            return False
        # Если встретили скобку, убираем ее из стека и начинаем разбирать выражение внутри скобок
        elif (st.top() == '('):
            st.pop()
            
            if (st.top() == 'N'):
                st.pop()
                return not self.__scan(st)
            # Если это не Not выражение, то получаем значение левой части, узнаем по символу А, O или X,
            # что это за выражение, и узнаем его правую часть, после чего возвращаем boolean значение
            else:
                isLgA, isLgO, isLgX = False, False, False
                
                # Левая часть 
                lgLeft = self.__scan(st)
                
                # Если левая часть была большой, то в конце могла остаться скобка, убираем ее
                if (st.top() == ')'):
                    st.pop()
                
                # Определяем, что это за выражение    
                if (st.top() == 'A'):
                    isLgA = True;
                elif (st.top() == 'O'):
                    isLgO = True;
                elif (st.top() == 'X'):
                    isLgX = True;
                st.pop()
                
                # Правая часть
                lgRight = self.__scan(st)
                
                if (isLgA):
                    # AND
                    return lgLeft == lgRight
                elif (isLgX):
                    # XOR
                    return lgLeft != lgRight
                elif (isLgO):
                    # OR
                    return lgLeft or lgRight

In [14]:
s0 = findLg('((NF)O(NF))')

print(s0.s, s0.res)

((NF)O(NF)) True


### Задание 10

Дан текстовый файл. В текстовом файле записана формула следующего вида:

<Формула> ::= <Цифра> | M(<Формула>,<Формула>) | N(Формула>,<Формула>)

< Цифра > ::= 0 | 1 | 2 | 3 | 4 | 5 | 6 | 7 | 8 | 9

где буквами обозначены функции:

M – определение максимума
N – определение минимума
Используя Stack, вычислить значение заданного выражения.

In [25]:
class findNum:
    def __init__(self, s):
        # Стек, в котором в дальнейшем происходит работа со строкой
        st = Stack(len(s))
        
        # Заполнение стека по-символьно содержимым строки
        for c in reversed(s):
            st.push(c)
        # В переменную res заносим результат, а исходную строку помещаем в s
        self.res = self.__scan(st)
        self.s = s
        
    # Основной метод
    def __scan(self, st):
        # Встретиили цифры
        if ord(st.top()) >= 48 and ord(st.top()) <= 57:
            return int(st.pop())
        
        # Встретили формулу максимума
        elif st.top() == 'M':
            st.pop() # Убираем букву
            st.pop() # Убираем скобку "("
            
            leftNum = self.__scan(st)
            
            # Если часть выражения была формулой, то убираем оставшуюся от него скобку
            if st.top() == ')':
                st.pop()
                
            st.pop() # Убираем ","
            
            rightNum = self.__scan(st)
            
            return max(leftNum, rightNum)
        # Встретили формулу минимума
        elif st.top() == 'N':
            st.pop() # Убираем букву
            st.pop() # Убираем скобку "("
            
            leftNum = self.__scan(st)
            
            # Если часть выражения была формулой, то убираем оставшуюся от него скобку
            if st.top() == ')':
                st.pop()
                
            st.pop() # Убираем ","
            
            rightNum = self.__scan(st)
            
            return min(leftNum, rightNum)

In [28]:
s0 = findNum('N(M(9,3),M(7,N(6,4)))')

print(s0.s, s0.res)

N(M(9,3),M(7,N(6,4))) 7


### Задание 11
Дан текстовый файл. Используя Stack, проверить, является ли содержимое текстового файла правильной записью формулы вида:

- < Формула > ::= < Терм > | < Терм > + < Формула > | < Терм > - < Формула >
- < Терм > ::= < Имя > | (< Формула >)
- < Имя > ::= x | y | z

In [17]:
class chForm:
    def __init__(self, s):
        # Стек, в котором в дальнейшем происходит работа со строкой
        st = Stack(len(s))
        
        # Заполнение стека по-символьно содержимым строки
        for c in reversed(s):
            st.push(c)
        # В переменную res заносим результат, а исходную строку помещаем в s
        self.res = self.__scan(st)
        self.s = s
        
    # Основной метод
    def __scan(self, st):
        # Если код символа из Stack`а с верхней позиции равен коду 'x', 'y' или 'z', то
        # Удаляем символ и возвращаем True
        if ord(st.top()) >= 120 and ord(st.top())<= 122:
            st.pop()
            return True
        
        # Если, встретили скобку, то пытаемся проверить на корректность выражение внутри 
        elif st.top() == '(':
            st.pop() # Убираем скобку
            
            leftForm = self.__scan(st) # провермяем левую часть выражения
            
            # Если часть выражения была формулой, то убираем оставшуюся от него скобку
            if st.top() == ')':
                st.pop()
            
            st.pop() # Удаляем пробел
            # Если знак по-середине выражения не является разрешенным, возвращаем False
            if st.top() != '-' and st.top() != '+':
                return False
            else:
                st.pop()
            st.pop()# Удаляем второй пробел
            
            rightForm = self.__scan(st)# Провермяем правую часть выражения
            
            # Если часть выражения была формулой, то убираем оставшуюся от него скобку
            if st.top() == ')':
                st.pop()
            
            # Если левая или правая часть вернула False, то возвращаем False
            if rightForm == False or leftForm == False:
                return False
            # Если Stack уже пуст, то возвращаем True
            elif st.isEmpty():
                return True
            
        # Если попался символ, которого нет в списке разрешенных, возвращаем False
        else:
            return False

In [24]:
s0 = chForm('(x + (y + (z + x)))')
s1 = chForm('((y - x) - z)')
s2 = chForm('((y + x) - f)')

print(s0.s, s0.res)
print(s1.s, s1.res)
print(s2.s, s2.res)

(x + (y + (z + x))) True
((y - x) - z) True
((y + x) - f) False


### Вывод
Выполнив эту лабораторную работу, я осознала принцип работы классов Stack и Deque.